# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel
Team: Red Bull Racing


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 World Championship was won by Lewis Hamilton driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Version 1 : Zero-Shot Prompting

In [186]:
context_zero_shot = [
    {'role': 'system', 'content':
     """
     Your task is to classify product reviews into three categories: Positive, Negative, or Neutral.
     Read the given review and decide the sentiment.

     """}
]


## Version 2 : One-Shot Prompting

In [187]:
context_one_shot = [
    {'role': 'system', 'content':
     """
     Your task is to classify product reviews into three categories: Positive, Negative, or Neutral.
     Example:
     - Review: "It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does."
       Sentiment: Negative
     """}
]

## Version 3 : Few-Shot Prompting

In [188]:
context_few_shot = [
    {'role': 'system', 'content': 
     """
     Your task is to classify product reviews into three categories: Positive, Negative, or Neutral.
    """},

    {'role': 'user', 'content': "The quality is excellent, I would definitely buy it again!"},
    {'role': 'assistant', 'content': 'Sentiment: Positive'},

    {'role': 'user', 'content': "It stopped working after just one week! Not worth the money."},
    {'role': 'assistant', 'content': 'Sentiment: Negative'},

    {'role': 'user', 'content': "It works fine, but nothing special. It's just okay."},
    {'role': 'assistant', 'content': 'Sentiment: Neutral'},

    {'role': 'user', 'content': "The price is good, but the quality could be better."},
    {'role': 'assistant', 'content': 'Sentiment: Neutral'},

    {'role': 'user', 'content': "The design is beautiful, but the battery drains too fast."},
    {'role': 'assistant', 'content': 'Sentiment: Negative'}
]


In [189]:
# Test sentences and their ground truth sentiments
test_cases = [
    {"sentence": "The quality is outstanding, but I had trouble with shipping delays.", "ground_truth": "Neutral"},
    {"sentence": "It stopped working after just one week! Not worth the money.", "ground_truth": "Negative"},
    {"sentence": "The design is nice, but the battery drains too fast.", "ground_truth": "Negative"},
    {"sentence": "It works as expected. Nothing special, but no major complaints.", "ground_truth": "Neutral"},
    {"sentence": "Fantastic product! The build quality and performance are excellent.", "ground_truth": "Positive"}
]

# Initialize counters for correct predictions
correct_zero_shot = 0
correct_one_shot = 0
correct_few_shot = 0

# Total test cases
total_cases = len(test_cases)

In [190]:
# Test Zero-Shot
print("Testing Zero-Shot...")
for test in test_cases:
    response = return_OAIResponse(test["sentence"], context_zero_shot)
    print(f"Input: {test['sentence']}")
    print(f"Model Output: {response}")
    print(f"Ground Truth: {test['ground_truth']}")
    print("---")
    if test["ground_truth"].lower() in response.lower():
        correct_zero_shot += 1

Testing Zero-Shot...
Input: The quality is outstanding, but I had trouble with shipping delays.
Model Output: The review contains both positive and negative sentiments. Overall, it seems to be a mixture of positive and negative feedback.
Ground Truth: Neutral
---
Input: It stopped working after just one week! Not worth the money.
Model Output: Negative
Ground Truth: Negative
---
Input: The design is nice, but the battery drains too fast.
Model Output: Negative
Ground Truth: Negative
---
Input: It works as expected. Nothing special, but no major complaints.
Model Output: Neutral
Ground Truth: Neutral
---
Input: Fantastic product! The build quality and performance are excellent.
Model Output: Sentiment: Positive
Ground Truth: Positive
---


In [191]:
# Test One-Shot
print("\nTesting One-Shot...")
for test in test_cases:
    response = return_OAIResponse(test["sentence"], context_one_shot)
    print(f"Input: {test['sentence']}")
    print(f"Model Output: {response}")
    print(f"Ground Truth: {test['ground_truth']}")
    print("---")
    if test["ground_truth"].lower() in response.lower():
        correct_one_shot += 1



Testing One-Shot...
Input: The quality is outstanding, but I had trouble with shipping delays.
Model Output: Sentiment: Neutral
Ground Truth: Neutral
---
Input: It stopped working after just one week! Not worth the money.
Model Output: Sentiment: Negative
Ground Truth: Negative
---
Input: The design is nice, but the battery drains too fast.
Model Output: Sentiment: Negative
Ground Truth: Negative
---
Input: It works as expected. Nothing special, but no major complaints.
Model Output: Sentiment: Neutral
Ground Truth: Neutral
---
Input: Fantastic product! The build quality and performance are excellent.
Model Output: Sentiment: Positive
Ground Truth: Positive
---


In [192]:
# Test Few-Shot
print("\nTesting Few-Shot...")
for test in test_cases:
    response = return_OAIResponse(test["sentence"], context_few_shot)
    print(f"Input: {test['sentence']}")
    print(f"Model Output: {response}")
    print(f"Ground Truth: {test['ground_truth']}")
    print("---")
    if test["ground_truth"].lower() in response.lower():
        correct_few_shot += 1


Testing Few-Shot...
Input: The quality is outstanding, but I had trouble with shipping delays.
Model Output: Sentiment: Neutral
Ground Truth: Neutral
---
Input: It stopped working after just one week! Not worth the money.
Model Output: Sentiment: Negative
Ground Truth: Negative
---
Input: The design is nice, but the battery drains too fast.
Model Output: Sentiment: Negative
Ground Truth: Negative
---
Input: It works as expected. Nothing special, but no major complaints.
Model Output: Sentiment: Neutral
Ground Truth: Neutral
---
Input: Fantastic product! The build quality and performance are excellent.
Model Output: Sentiment: Positive
Ground Truth: Positive
---


In [193]:
# Calculate and print accuracy for each context
accuracy_zero_shot = (correct_zero_shot / total_cases) * 100
accuracy_one_shot = (correct_one_shot / total_cases) * 100
accuracy_few_shot = (correct_few_shot / total_cases) * 100

print("\nAccuracy Results:")
print(f"Zero-Shot: {accuracy_zero_shot:.2f}% ({correct_zero_shot}/{total_cases} correct)")
print(f"One-Shot: {accuracy_one_shot:.2f}% ({correct_one_shot}/{total_cases} correct)")
print(f"Few-Shot: {accuracy_few_shot:.2f}% ({correct_few_shot}/{total_cases} correct)")


Accuracy Results:
Zero-Shot: 80.00% (4/5 correct)
One-Shot: 100.00% (5/5 correct)
Few-Shot: 100.00% (5/5 correct)


# 📝 Evaluation of Different Three Versions  


## 📌 Overview
This report evaluates the performance of three different prompting approaches for classifying product reviews: Zero-Shot, One-Shot, and Few-Shot. The focus is on accuracy, clarity, and the ability to handle complex sentiments based on experimental results.

---

## Version 1: (Zero-Shot Prompting)
### ✅ Pros:
* Simple and fast to implement.
* Relies solely on clear instructions without examples.
* Works well for straightforward tasks.
### ❌ Cons:
* Struggles with ambiguous or complex sentiments.
* Lacks examples to guide the model.
### 📊 Accuracy: **80.00%** (4/5 correct)

---
## Version 2: (One-Shot Prompting)
### ✅ Pros: 
* Provides a single example to guide the model.
* Significantly improves accuracy compared to Zero-Shot.
* Useful for tasks with a clear pattern.
### ❌ Cons:
* Limited context; one example may not cover all cases.
### 📊 Accuracy: **100.00%** (5/5 correct)

---

## Version 3: (Few-Shot Prompting)
### ✅ Pros:
* Offers multiple examples, covering a range of sentiments (Positive, Negative, Neutral).
* Provides rich context, improving the model's understanding.
* Handles complex or ambiguous reviews more effectively.
### ❌ Cons:
* Slightly more effort to set up due to the need for multiple examples.
### 📊 Accuracy: **100.00%** (5/5 correct)

---

## 🎯 Conclusion & Key Learnings
* **One-Shot and Few-Shot Prompting both achieved perfect accuracy (100%), demonstrating their effectiveness in sentiment classification.**
* **Few-Shot Prompting provides a more robust framework, especially for handling nuanced or ambiguous reviews.**
* **Through this experiment, we learned that providing context significantly improves model performance, and adding diverse examples helps handle edge cases more effectively.**




